In [ ]:
%load_ext tutormagic

# Tree Class

We previously represented trees using data abstraction. Now we'll write a class for that.

## Tree Abstraction (Review)

A tree is a root label and a list of branches. Recall that we have 2 sets of vocabulary to describe the same abstraction.

#### Recursive description (wooden trees):
1. A `tree` has a `root label` and a list of `branches`
2. Each `branch` is a `tree`
3. A `tree` with 0 `branches` is called a `leaf`
4. A `tree` starts at the `root`

<img src = 'recursive.jpg' width = 500/>

#### Relative description (family trees):
1. Each location in a tree is called a `node`
    * Our implementation of trees describes labels and branches, but nodes are used to describe the characteristic of trees
        * e.g. how many labels appear in the whole thing
2. Each `node` has a `label` that can be any value
3. A node can be the `parent/child` of another
    * The `node` with `3` is the parent of the `node` with `2`
    * The `node` with `1` is the grandchild of the `node` with 3
4. The top node is the `root node`

<img src = 'relative.jpg' width = 600/>|

A sequence of nodes from the root to a leaf is called a `path`. Below is an example of a path from 3 to 0 (brown lines).

<img src = 'path.jpg' width = 600/>

## Tree Class

A Tree has a label and a list of branches (by default, it's empty). Each branch is a Tree. Below is the class implementation of `Tree`.

In [ ]:
class Tree:
    def __init__(self, label, branches = []):
        self.label = label
        # Makes sure that each branch is an instance of the tree class
        for branch in branches:
            assert isintance(branch, Tree) 
        # Set the branches to a list version of the branches
        self.branches = list(branches)

Recall the tree implementation using data abstraction,

In [2]:
def tree(label, branches = []):
    for branch in branches:
        assert is_tree(branch)
    return [label] + list(branches)

def label(tree):
    return tree[0]

def branches(tree):
    return tree[1:]

The implementation using data abstraction requires more work! Not only we have to define the constructor (`tree`), we also need to define the selectors (`label` `branches`). More importantly, we needed to invent a representation! Notice the last line of the `tree` function:

In [ ]:
return [label] + list(branches)

We decided that a `Tree` would be represented as a list containing a label (`[label]`) and whatever's left as the branches. This way, the selectors has to match the constructors. 

The details above are not necessary when using the object system. We only need to declare what the attributes are for each instance of the `Tree` class. On top of that, we know how to access them via dot `.` notation.

In [3]:
def fib_tree(n):
    if n == 0 or n == 1:
        return Tree(n)
    else:
        left = fib_tree(n-2)
        right = fib_tree(n-1)
        fib_n = left.label + right.label
        return Tree(fib_n, [left, right])

Above is an implementation of a function that constructs the Fibonacci tree using the `Tree` class. Compare it with the implementation if we were to use data abstraction, 

In [4]:
def fib_tree(n):
    if n == 0 or n == 1:
        return tree(n)
    else:
        left = fib_tree(n-2)
        right = fib_tree(n-1)
        fib_n = label(left) + label(right)
        return tree(fib_n, [left, right])

Both implementations are very similar, with the only differences that the class implementation uses capital `T` and uses the dot `.` notation for accessing the label.

## Demo

Below is our `Tree` class,

In [11]:
class Tree:
    """A tree is a label and a list of branches."""
    def __init__(self, label, branches=[]):
        self.label = label
        for branch in branches:
            assert isinstance(branch, Tree)
        self.branches = list(branches)

    def __repr__(self):
        if self.branches:
            branch_str = ', ' + repr(self.branches)
        else:
            branch_str = ''
        return 'Tree({0}{1})'.format(repr(self.label), branch_str)

    def __str__(self):
        return '\n'.join(self.indented())

    def indented(self):
        lines = []
        for b in self.branches:
            for line in b.indented():
                lines.append('  ' + line)
        return [str(self.label)] + lines
    
    def is_leaf(self):
        return not self.branches

In [2]:
Tree(2) # Construct a leaf

Tree(2)

In [3]:
Tree(2, [3])

AssertionError: 

Above, we tried to construct a `Tree` with a branch that is not a `Tree`, which didn't work and gave us an error message. A branch has to be a `Tree`. 

In [4]:
Tree(2, [Tree(3)])

Tree(2, [Tree(3)])

In [5]:
Tree(2, [Tree(3), Tree(4)])

Tree(2, [Tree(3), Tree(4)])

If we `print` the tree above,

In [6]:
print(Tree(2, [Tree(3), Tree(4)]))

2
  3
  4


We have 2 as the root, 3 and 4 as the branches. 

Below we have the memoization function and the `fib_tree` function.

In [12]:
def memo(f):
    cache = {}
    def memoized(n):
        if n not in cache:
            cache[n] = f(n)
        return cache[n]
    return memoized

@memo
def fib_tree(n):
    """A Fibonacci tree.

    >>> print(fib_tree(4))
    3
      1
        0
        1
      2
        1
        1
          0
          1
    """
    if n == 0 or n == 1:
        return Tree(n)
    else:
        left = fib_tree(n-2)
        right = fib_tree(n-1)
        fib_n = left.label + right.label
        return Tree(fib_n, [left, right])

In [8]:
fib_tree(4)

Tree(3, [Tree(1, [Tree(0), Tree(1)]), Tree(2, [Tree(1), Tree(1, [Tree(0), Tree(1)])])])

As we can see, it is difficult to read the output in its `__repr__` string. However, if we print it out,

In [9]:
print(fib_tree(4))

3
  1
    0
    1
  2
    1
    1
      0
      1


Then we can see the structure clearly! 

Even though we're using a class to represent the Fibonacci tree, we can still construct large trees using memoization.

In [10]:
t = fib_tree(100)

However, if we try to print `t`, it will take a while since the printed representation of the tree has a lot of redundancy. 

Processing a tree that's an instance of a tree class is almost identical to processing a tree using data abstraction. For example, if we want to write a function that returns a list of the leaves in a tree, 

In [1]:
def leaves(tree):
    if tree.is_leaf():
        return [tree.label] #Return a tree containing the label of that leaf
    else:
        s = [] #Construct a list of all the leaves labels
        for b in tree.branches:
            s.extend(leaves(b)) # Extend s with the result of recursive call of leaves on b
        return s

In [14]:
leaves(fib_tree(8))

[0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1]